# Daily Wave Stats with 4 parameters

In [35]:
!pip install pandas
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()

Saving Boya_Tenerife_Sur_4parameters.csv.csv to Boya_Tenerife_Sur_4parameters.csv.csv


In [ ]:
# Recogemos el primer archivo cargado
filename = list(uploaded.keys())[0]

# Leemos el .csv
df = pd.read_csv(io.BytesIO(uploaded[filename]), sep=';', decimal=',', encoding='latin-1')

# Primeras filas del DataFrame
print(df.head())
# Obtenemos información sobre las columnas y tipos de datos
print(df.info())


           Fecha  Altura  Periodo de pico  Dir. Media de Procedencia
0  2003 05 06 09    1.26             4.64                         90
1  2003 05 06 10    1.17             4.53                         88
2  2003 05 06 11    1.16             4.99                         88
3  2003 05 06 12    1.23             4.87                         89
4  2003 05 06 13    1.22            14.26                         87
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163887 entries, 0 to 163886
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Fecha                      163887 non-null  object 
 1   Altura                     163887 non-null  float64
 2   Periodo de pico            163887 non-null  float64
 3   Dir. Media de Procedencia  163887 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 5.0+ MB
None


Vamos a crear un nuevo .csv del archivo 'Boya_Tenerife_Sur_4parameters' en el que haremos el max, mean y min de cada parámetro (Altura (m), Periodo de Pico (s) y Dir. Media de Procedencia (º)), de manera diaria.

In [ ]:
# Le quitamos la hora porque lo que queremos hacer es un groupby y ya ta
df['Fecha'] = pd.to_datetime(df['Fecha'])  # Convertimos 'Fecha' en datetime
df['Fecha'] = df['Fecha'].dt.strftime('%Y-%m-%d') # Le quito la hora
df['Fecha'] = pd.to_datetime(df['Fecha']) # Convertimos en datetime64[ns]

print(df.head())

       Fecha  Altura  Periodo de pico  Dir. Media de Procedencia
0 2003-05-06    1.26             4.64                         90
1 2003-05-06    1.17             4.53                         88
2 2003-05-06    1.16             4.99                         88
3 2003-05-06    1.23             4.87                         89
4 2003-05-06    1.22            14.26                         87


In [ ]:
# Convertir las columnas numéricas asegurando el formato correcto
df[['Altura', 'Periodo de pico', 'Dir. Media de Procedencia']] = df[['Altura', 'Periodo de pico', 'Dir. Media de Procedencia']].apply(pd.to_numeric, errors='coerce')

# Antes de hacer el promedio ni nada, quitemos las negativas y nulas
df = df.dropna()
df = df[df['Altura'] >= 0]
df = df[df['Periodo de pico'] >= 0]
df = df[df['Dir. Media de Procedencia'] >= 0]

# Agrupamos por fecha y calculamos estadísticas
df_daily = df.groupby('Fecha').agg({
    'Altura': ['max', 'mean', 'min'],
    'Periodo de pico': ['max', 'mean', 'min'],
    'Dir. Media de Procedencia': ['max', 'mean', 'min']
}).reset_index()

# Renombramos columnas resultantes
df_daily.columns = [
    'Fecha', 'hmax (m)', 'hmean (m)', 'hmin (m)',
    'Tpico_max (s)', 'Tpico_mean (s)', 'Tpico_min (s)',
    'Dir_procedencia_max (º)', 'Dir_procedencia_mean (º)', 'Dir_procedencia_min (º)'
]

In [39]:
# Guardar el resultado en un CSV
df_daily.to_csv('DailyWaveStats_4p_20_23_TS.csv', index=False, sep=';', decimal=',')

# Mostrar las primeras filas del DataFrame procesado
print(df_daily.head())

       Fecha  hmax (m)  hmean (m)  hmin (m)  Tpico_max (s)  Tpico_mean (s)  \
0 2003-05-06      1.76   1.415333      1.16          14.26        7.235333   
1 2003-05-07      1.77   1.502083      1.34           6.65        5.754167   
2 2003-05-08      1.68   1.464583      1.21           6.24        5.517917   
3 2003-05-09      1.51   1.219167      1.01           5.87        5.013750   
4 2003-05-10      1.39   1.142917      0.83          12.48        8.748750   

   Tpico_min (s)  Dir_procedencia_max (º)  Dir_procedencia_mean (º)  \
0           4.53                       92                 86.533333   
1           5.27                       94                 88.166667   
2           4.99                       94                 89.333333   
3           3.83                       95                 88.625000   
4           4.53                      103                 92.125000   

   Dir_procedencia_min (º)  
0                       82  
1                       83  
2                

## Fechas faltantes

In [ ]:
# Generar la secuencia completa de fechas desde el 2000-01-01 hasta el 2023-12-31
full_date_range = pd.date_range(start="2000-01-01", end="2023-12-31")

# Convertir la columna de fechas en datetime
df_daily['Fecha'] = pd.to_datetime(df_daily['Fecha'])

# Encontrar fechas que faltan comparando con la secuencia completa
missing_dates = full_date_range.difference(df_daily['Fecha'])

# Mostrar las primeras fechas faltantes (si hay muchas)
print("Fechas faltantes:", missing_dates)

# Guardar las fechas faltantes en un CSV si quieres analizarlas después
missing_dates_4p_df = pd.DataFrame(missing_dates, columns=["Fechas Faltantes"])
missing_dates_4p_df.to_csv("MD_DailyWaveStats_4p_20_23_TS.csv", index=False)

Fechas faltantes: DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10',
               ...
               '2021-05-10', '2021-05-11', '2023-11-06', '2023-11-07',
               '2023-11-08', '2023-11-09', '2023-11-10', '2023-11-11',
               '2023-11-12', '2023-11-13'],
              dtype='datetime64[ns]', length=1887, freq=None)
